In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d aashita/nyt-comments

kaggle.json
 99% 474M/480M [00:05<00:00, 60.5MB/s]
100% 480M/480M [00:05<00:00, 92.4MB/s]


In [ ]:
!unzip /content/nyt-comments.zip

Archive:  /content/nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [ ]:
import pandas as pd
import glob
articles = glob.glob('/content/Articles*.csv')

In [ ]:
li = []

for filename in articles:
    df_a = pd.read_csv(filename, index_col=None, header=0)
    li.append(df_a)

articles = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
articles.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5adb8334068401528a2a9f74,1571,By THE EDITORIAL BOARD,article,The Opioid Crisis Foretold,"['Opioids and Opiates', 'Drug Abuse and Traffi...",68,Editorial,10,2018-04-21 18:30:10,Sunday Review,History supplies precedents and lessons for th...,The New York Times,Editorial,https://www.nytimes.com/2018/04/21/opinion/an-...
1,NaN,5adb8331068401528a2a9f70,1221,By PETER FRITSCH and GLENN R. SIMPSON,article,The Business Deals That Could Imperil Trump,['Russian Interference in 2016 US Elections an...,68,OpEd,3,2018-04-21 18:30:06,Sunday Review,Forget Stormy Daniels. The business records su...,The New York Times,Op-Ed,https://www.nytimes.com/2018/04/21/opinion/sun...
2,NaN,5adb8330068401528a2a9f6f,1465,By CHRISTOPHER A. PREBLE,article,Adapting to American Decline,"['United States International Relations', 'Uni...",68,OpEd,1,2018-04-21 18:30:05,Sunday Review,The United States is losing its influence as i...,The New York Times,Op-Ed,https://www.nytimes.com/2018/04/21/opinion/sun...
3,NaN,5adb8330068401528a2a9f6d,1097,By FRANK BRUNI,article,The Republicans’ Big Senate Mess,"['United States Politics and Government', 'Ele...",67,OpEd,3,2018-04-21 18:30:04,Sunday Review,"A convict in West Virginia, bondage in Missour...",The New York Times,Op-Ed,https://www.nytimes.com/2018/04/21/opinion/sun...
4,NaN,5adb8330068401528a2a9f6e,1306,By SHARON LERNER,article,States Are Doing What Scott Pruitt Won’t,"['Water Pollution', 'Hazardous and Toxic Subst...",68,OpEd,2,2018-04-21 18:30:04,Sunday Review,The federal government may ignore the environm...,The New York Times,Op-Ed,https://www.nytimes.com/2018/04/21/opinion/sun...


In [ ]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9335 entries, 0 to 9334
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   abstract          167 non-null    object
 1   articleID         9335 non-null   object
 2   articleWordCount  9335 non-null   int64 
 3   byline            9335 non-null   object
 4   documentType      9335 non-null   object
 5   headline          9335 non-null   object
 6   keywords          9335 non-null   object
 7   multimedia        9335 non-null   int64 
 8   newDesk           9335 non-null   object
 9   printPage         9335 non-null   int64 
 10  pubDate           9335 non-null   object
 11  sectionName       9335 non-null   object
 12  snippet           9335 non-null   object
 13  source            9335 non-null   object
 14  typeOfMaterial    9335 non-null   object
 15  webURL            9335 non-null   object
dtypes: int64(3), object(13)
memory usage: 1.1+ MB


In [ ]:
#Analyzing the data
print(articles['sectionName'].nunique()) 
print(articles['sectionName'].unique())

62
['Sunday Review' 'Unknown' 'Politics' 'Asia Pacific' 'Baseball' 'Mind'
 'Move' 'Golf' 'Eat' 'Live' 'Lesson Plans' 'Television' 'Family'
 'DealBook' "401(k)'s and Similar Plans" 'Retirement' 'Economy' 'Africa'
 'Soccer' 'Pro Basketball' 'Europe' 'Rugby' 'College Basketball'
 'Middle East' 'Pro Football' 'Energy & Environment ' 'Art & Design'
 'Personal Tech' 'College Football' 'Music' 'Media' 'Hockey'
 'Paying for College' 'Olympics' 'Tennis' 'Insider Events' 'Australia'
 'Wine, Beer & Cocktails' 'Opinion | Politics' 'Americas' 'The Daily'
 'Canada' 'Book Review' 'Automobiles' 'Food' 'Dance' 'Art'
 'Opinion | The World' 'Room For Debate' 'Education Life' 'Editorials'
 'Fashion & Beauty' 'Entrepreneurship' 'Entertainment' 'Weddings'
 'Real Estate' 'Neighborhoods' 'Cricket' 'Student Loans' 'Auto Racing'
 'Cycling' 'Learning']


In [ ]:
articles['snippet']

0       History supplies precedents and lessons for th...
1       Forget Stormy Daniels. The business records su...
2       The United States is losing its influence as i...
3       A convict in West Virginia, bondage in Missour...
4       The federal government may ignore the environm...
                              ...                        
9330    Several states and cities have ordered employe...
9331    The beginning, middle and end of a picture boo...
9332    The former N.S.A. and C.I.A. chief on Trump, S...
9333    In “Something Wonderful,” Todd S. Purdum analy...
9334    A pair of two-part productions, “Harry Potter ...
Name: snippet, Length: 9335, dtype: object

In [ ]:
art = articles[['articleID', 'headline',	'keywords', 'snippet']]

In [ ]:
# Importing the pachages I need
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import RegexpTokenizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel #LDA model

In [ ]:
# Lemmatization
from nltk.stem.wordnet import WordNetLemmatizer

# Enabling logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
import nltk
from nltk.corpus import stopwords 
import string
nltk.download('stopwords')
nltk.download('punkt')
stop_words_en = stopwords.words("english") 

stop_words_en = stop_words_en + list(string.punctuation)
stop_words_en = stopwords.words("english") 

stop_words_en = stop_words_en + list(string.punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Cleaning the dataset
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc = True removes punctuations

data_words = list(sent_to_words(art['snippet']))

print(data_words[:1])

[['history', 'supplies', 'precedents', 'and', 'lessons', 'for', 'the', 'opioid', 'epidemic', 'raging', 'in', 'america', 'today', 'but', 'we', 'seem', 'to', 'have', 'largely', 'forgotten', 'both']]


In [ ]:
# Defining functions for stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_en] for doc in texts]

In [ ]:
# Splitting the documents into tokens
tokenizer = RegexpTokenizer(r'\w+')

for idx in range(len(art['keywords'])):
    art['keywords'][idx] = art['keywords'][idx]
    art['keywords'][idx] = tokenizer.tokenize(str(art['keywords'][idx]) ) # Split into words.

# Removing numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in art['keywords']]

# Removing words that are only one character
docs = [[token for token in doc if len(token) > 1] for doc in art['keywords']]

docs = [[word for word in doc if word not in stop_words_en] for doc in art['keywords']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
nltk.download('wordnet')
# Considering only words that satisfy the prerequisites previously described
lemmatizer = WordNetLemmatizer()
data_lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Ensuring that all the stop words and punctuations are deleted
data_words_nostops = remove_stopwords(data_lemmatized)
data_words_nostops

[['opioids',
  'opiates',
  'drug',
  'abuse',
  'traffic',
  'united',
  'states',
  'politics',
  'government',
  'epidemics',
  'acquired',
  'immune',
  'deficiency',
  'syndrome',
  'purdue',
  'pharma'],
 ['russian',
  'interference',
  'us',
  'elections',
  'ties',
  'trump',
  'associates',
  'presidential',
  'election',
  'money',
  'laundering',
  'real',
  'estate',
  'commercial',
  'foreign',
  'corrupt',
  'practices',
  'act',
  'trump',
  'organization',
  'trump',
  'international',
  'hotel',
  'tower',
  'panama',
  'city',
  'panama',
  'trump',
  'soho',
  'manhattan',
  'ny',
  'hotel',
  'cohen',
  'michael',
  'mueller',
  'robert',
  'iii',
  'steele',
  'christopher',
  'trump',
  'donald'],
 ['united',
  'states',
  'international',
  'relations',
  'united',
  'states',
  'defense',
  'military',
  'forces',
  'south',
  'korea'],
 ['united', 'states', 'politics', 'government', 'elections', 'senate'],
 ['water',
  'pollution',
  'hazardous',
  'toxic',
  '

In [ ]:
# Creating Dictionary
id2word = gensim.corpora.Dictionary(data_words_nostops)

In [ ]:
# Creating Corpus for my function
texts = data_words_nostops

In [ ]:
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(texts[0])
print(corpus[0])

['opioids', 'opiates', 'drug', 'abuse', 'traffic', 'united', 'states', 'politics', 'government', 'epidemics', 'acquired', 'immune', 'deficiency', 'syndrome', 'purdue', 'pharma']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [ ]:
# Building the function I'm going to use in order to perform the topic model
model = gensim.models.LdaModel(corpus=corpus,
                               id2word=id2word,
                               chunksize=100,
                               alpha=0.1,
                               eta=0.001,
                               random_state=123,
                               num_topics=62)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [ ]:
model.show_topic(0)

[('inc', 0.92693055),
 ('diet', 0.029002342),
 ('nutrition', 0.029002342),
 ('education', 1.3934357e-06),
 ('back', 1.3933817e-06),
 ('city', 1.3933807e-06),
 ('nyc', 1.393379e-06),
 ('roots', 1.3933749e-06),
 ('department', 1.3933737e-06),
 ('company', 1.3933723e-06)]

In [ ]:
model.show_topic(1)

[('george', 0.35008472),
 ('bush', 0.25227243),
 ('barbara', 0.11544068),
 ('stress', 0.06764841),
 ('weight', 0.058209155),
 ('anxiety', 0.052729107),
 ('obesity', 0.036252446),
 ('reagan', 0.02823805),
 ('nancy', 0.016401164),
 ('hormone', 3.0484829e-05)]

In [ ]:
pprint(model.print_topics())

[(55,
  '0.339*"tv" + 0.339*"program" + 0.232*"television" + 0.042*"reading" + '
  '0.042*"skills" + 0.000*"legion" + 0.000*"education" + 0.000*"writing" + '
  '0.000*"beauvoir" + 0.000*"pencils"'),
 (10,
  '0.392*"women" + 0.383*"girls" + 0.080*"running" + 0.058*"sarah" + '
  '0.042*"boston" + 0.025*"adam" + 0.009*"grant" + 0.000*"marathon" + '
  '0.000*"trump" + 0.000*"election"'),
 (14,
  '0.176*"privacy" + 0.164*"group" + 0.132*"sports" + 0.120*"systems" + '
  '0.090*"science" + 0.050*"technology" + 0.042*"joe" + 0.041*"tennessee" + '
  '0.035*"field" + 0.028*"williams"'),
 (16,
  '0.131*"trump" + 0.118*"us" + 0.117*"russian" + 0.114*"ties" + '
  '0.114*"associates" + 0.114*"interference" + 0.091*"robert" + 0.062*"cohen" '
  '+ 0.050*"mueller" + 0.026*"organization"'),
 (5,
  '0.546*"league" + 0.315*"baseball" + 0.090*"major" + 0.000*"crossword" + '
  '0.000*"puzzles" + 0.000*"fusion" + 0.000*"deneuve" + 0.000*"pencil" + '
  '0.000*"belleclaire" + 0.000*"dreams"'),
 (43,
  '0.718*"

In [ ]:
doc_lda = model[corpus]
print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -485.01869504326305

Coherence Score:  0.46792104177404453


In [ ]:
def format_topics_sentences(ldamodel=model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,57.0,0.3352,"abuse, drug, traffic, opioids, opiates, epidem...","[opioids, opiates, drug, abuse, traffic, unite..."
1,1,16.0,0.4658,"trump, us, russian, ties, associates, interfer...","[russian, interference, us, elections, ties, t..."
2,2,15.0,0.6453,"international, united, relations, states, defe...","[united, states, international, relations, uni..."
3,3,34.0,0.3361,"government, politics, states, united, educatio...","[united, states, politics, government, electio..."
4,4,20.0,0.3592,"immune, water, buildings, sites, historic, haz...","[water, pollution, hazardous, toxic, substance..."
5,5,1.0,0.5000,"george, bush, barbara, stress, weight, anxiety...","[bush, barbara, bush, george, reagan, nancy, k..."
6,6,42.0,0.4671,"blacks, discrimination, race, police, ethnicit...","[race, ethnicity, blacks, discrimination, whit..."
7,7,48.0,0.1430,"puzzles, crossword, beauvoir, fusion, deneuve,...","[crossword, puzzles]"
8,8,24.0,0.5198,"state, agency, telephones, protection, scott, ...","[politics, government, lobbying, lobbyists, at..."
9,9,22.0,0.3821,"news, media, war, social, facebook, times, afg...","[facebook, inc, sri, lanka, demonstrations, pr..."


In [ ]:
df_dominant_topic['articleID'] = articles['articleID'].values
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,articleID
0,0,57.0,0.3352,"abuse, drug, traffic, opioids, opiates, epidem...","[opioids, opiates, drug, abuse, traffic, unite...",5adb8334068401528a2a9f74
1,1,16.0,0.4658,"trump, us, russian, ties, associates, interfer...","[russian, interference, us, elections, ties, t...",5adb8331068401528a2a9f70
2,2,15.0,0.6453,"international, united, relations, states, defe...","[united, states, international, relations, uni...",5adb8330068401528a2a9f6f
3,3,34.0,0.3361,"government, politics, states, united, educatio...","[united, states, politics, government, electio...",5adb8330068401528a2a9f6d
4,4,20.0,0.3592,"immune, water, buildings, sites, historic, haz...","[water, pollution, hazardous, toxic, substance...",5adb8330068401528a2a9f6e
...,...,...,...,...,...,...
9330,9330,10.0,0.1875,"women, girls, running, sarah, boston, adam, gr...","[discrimination, wages, salaries, labor, jobs,...",5ae82c93068401528a2ab969
9331,9331,53.0,0.3106,"books, literature, france, writing, writers, e...","[friendship, dewdney, anna, writing, writers, ...",5ae82c95068401528a2ab96b
9332,9332,16.0,0.3811,"trump, us, russian, ties, associates, interfer...","[classified, information, state, secrets, russ...",5ae82c9d068401528a2ab96d
9333,9333,53.0,0.2283,"books, literature, france, writing, writers, e...","[books, literature, purdum, todd, theater, ham...",5ae82c9f068401528a2ab96f
